# Document retrieval from wikipedia data Using TF-IDF

## Fire up GraphLab Create

In [2]:
import graphlab

# Load some text data - from wikipedia, pages on people

In [7]:
people = graphlab.SFrame('people_wiki.gl/')

### Let's View data set with head to view some of the rows let's 5 rows

Data contains:  link to wikipedia article, name of person, text of article.

In [8]:
people.head(5)

URI,name,text
<http://dbpedia.org/resource/Digby_Morrell> ...,Digby Morrell,digby morrell born 10october 1979 is a former ...
<http://dbpedia.org/resource/Alfred_J._Lewy> ...,Alfred J. Lewy,alfred j lewy aka sandylewy graduated from ...
<http://dbpedia.org/resource/Harpdog_Brown> ...,Harpdog Brown,harpdog brown is a singerand harmonica player who ...
<http://dbpedia.org/resource/Franz_Rottensteiner> ...,Franz Rottensteiner,franz rottensteiner bornin waidmannsfeld lower ...
<http://dbpedia.org/resource/G-Enka> ...,G-Enka,henry krvits born 30december 1974 in tallinn ...


In [9]:
len(people)

59071

# Explore the dataset and checkout the text it contains

## Exploring the entry for president Obama, Let's take subset of data as obama

In [10]:
obama = people[people['name'] == 'Barack Obama']

## obama variable contain all the information about Barack Obama from wiki

In [11]:
obama

URI,name,text
<http://dbpedia.org/resource/Barack_Obama> ...,Barack Obama,barack hussein obama iibrk husen bm born august ...


In [14]:
obama['text']

## Exploring the entry for actor George Clooney, in variable clooney

In [15]:
clooney = people[people['name'] == 'George Clooney']
clooney['text']

## Get the word counts for Obama article and store result in same data set as a new variable name word_count 

In [16]:
obama['word_count'] = graphlab.text_analytics.count_words(obama['text'])

In [17]:
print obama['word_count']

[{'operations': 1, 'represent': 1, 'office': 2, 'unemployment': 1, 'doddfrank': 1, 'over': 1, 'unconstitutional': 1, 'domestic': 2, 'major': 1, 'years': 1, 'against': 1, 'proposition': 1, 'seats': 1, 'graduate': 1, 'debate': 1, 'before': 1, 'death': 1, '20': 2, 'taxpayer': 1, 'representing': 1, 'obamacare': 1, 'barack': 1, 'to': 14, '4': 1, 'policy': 2, '8': 1, 'he': 7, '2011': 3, '2010': 2, '2013': 1, '2012': 1, 'bin': 1, 'then': 1, 'his': 11, 'march': 1, 'gains': 1, 'cuba': 1, 'school': 3, '1992': 1, 'new': 1, 'not': 1, 'during': 2, 'ending': 1, 'continued': 1, 'presidential': 2, 'states': 3, 'husen': 1, 'osama': 1, 'californias': 1, 'equality': 1, 'prize': 1, 'lost': 1, 'made': 1, 'inaugurated': 1, 'january': 3, 'university': 2, 'rights': 1, 'july': 1, 'gun': 1, 'stimulus': 1, 'rodham': 1, 'troop': 1, 'withdrawal': 1, 'brk': 1, 'nine': 1, 'where': 1, 'referred': 1, 'affordable': 1, 'attorney': 1, 'on': 2, 'often': 1, 'senate': 3, 'regained': 1, 'national': 2, 'creation': 1, 'related

### Sort the word counts for the Obama article, for sorting we use stack with new column name as word and count

#### Turning dictonary of word counts into a table

In [18]:
obama_word_count_table = obama[['word_count']].stack('word_count', new_column_name = ['word','count'])

### Sorting the word counts to show most common words at the top

In [22]:
obama_word_count_table.head(5)

word,count
normalize,1
sought,1
combat,1
continued,1
unconstitutional,1


In [23]:
obama_word_count_table.sort('count',ascending=False)

word,count
the,40
in,30
and,21
of,18
to,14
his,11
obama,9
act,8
a,7
he,7


Most common words include uninformative words like "the", "in", "and",...but some of the words does't contain any meaning full informaion about these article we can remove these words some time we call these words as stop words

# Compute TF-IDF for the corpus 

To give more weight to informative words, we weigh them by their TF-IDF scores. TF-IDF basically way to score the importance of words in document based on how frequently they appear across multiple doc, but this will work for the whole corpus so we apply tf-idf to the whole 

In [28]:
people['word_count'] = graphlab.text_analytics.count_words(people['text'])
people.head(5)

Columns:
	URI	str
	name	str
	text	str
	word_count	dict
	tfidf	dict

Rows: 5

Data:
+-------------------------------+---------------------+
|              URI              |         name        |
+-------------------------------+---------------------+
| <http://dbpedia.org/resour... |    Digby Morrell    |
| <http://dbpedia.org/resour... |    Alfred J. Lewy   |
| <http://dbpedia.org/resour... |    Harpdog Brown    |
| <http://dbpedia.org/resour... | Franz Rottensteiner |
| <http://dbpedia.org/resour... |        G-Enka       |
+-------------------------------+---------------------+
+-------------------------------+-------------------------------+
|              text             |           word_count          |
+-------------------------------+-------------------------------+
| digby morrell born 10 octo... | {'since': 1, 'carltons': 1... |
| alfred j lewy aka sandy le... | {'precise': 1, 'thomas': 1... |
| harpdog brown is a singer ... | {'just': 1, 'issued': 1, '... |
| franz rottensteiner born i... | {'all': 1, 'bauforschung':... |
| henry krvits born 30 decem... | {'legendary': 1, 'gangster... |
+-------------------------------+-------------------------------+
+-------------------------------+
|             tfidf             |
+-------------------------------+
| {'since': 1.45537671730804... |
| {'precise': 6.443200606955... |
| {'just': 2.700729968710864... |
| {'all': 1.6431112434912472... |
| {'legendary': 4.2808562943... |
+-------------------------------+
[5 rows x 5 columns]

In [30]:
tfidf = graphlab.text_analytics.tf_idf(people['word_count'])
tfidf

docs
"{'since':1.455376717308041, ..."
"{'precise':6.44320060695519, ..."
"{'just':2.7007299687108643, ..."
"{'all':1.6431112434912472, ..."
"{'legendary':4.280856294365192, ..."
"{'now': 1.96695239252401,'currently': ..."
"{'exclusive':10.455187230695827, ..."
"{'taxi':6.0520214560945025, ..."
"{'houston':3.935505942157149, ..."
"{'phenomenon':5.750053426395245, ..."


## add tf-idf docs to people dataset with a new variable called tfidf

In [31]:
people['tfidf'] = tfidf['docs']

## Examine the TF-IDF for the Obama article, this new variable with tfidf

In [32]:
obama = people[people['name'] == 'Barack Obama']

In [33]:
obama[['tfidf']].stack('tfidf',new_column_name=['word','tfidf']).sort('tfidf',ascending=False)

word,tfidf
obama,43.2956530721
act,27.678222623
iraq,17.747378588
control,14.8870608452
law,14.7229357618
ordered,14.5333739509
military,13.1159327785
involvement,12.7843852412
response,12.7843852412
democratic,12.4106886973


### Words with highest TF-IDF are much more informative. so we sorted words according to the TF-IDF 

# Manually compute distances between a few people

Let's manually compare the distances between the articles for a few famous people.  

In [34]:
clinton = people[people['name'] == 'Bill Clinton']

In [35]:
beckham = people[people['name'] == 'David Beckham']

### Is Obama closer to Clinton than to Beckham?

We will use cosine distance, which is given by

(1-cosine_similarity) for computing the distance between the two documents

and find that the article about president Obama is closer to the one about former president Clinton than that of footballer David Beckham.

In [36]:
graphlab.distances.cosine(obama['tfidf'][0],clinton['tfidf'][0])

0.8339854936884276

In [38]:
graphlab.distances.cosine(obama['tfidf'][0],beckham['tfidf'][0])

0.9791305844747478

#### less the distance between two document more they are similar two each other like obam and clinton are more similar then obama and beckham

# Build a nearest neighbor model for document retrieval

We now create a nearest-neighbors model and apply it to document retrieval.  

In [40]:
knn_model = graphlab.nearest_neighbors.create(people,features=['tfidf'],label='name')

PROGRESS: Starting brute force nearest neighbors model training.


# Applying the nearest-neighbors model for retrieval

## Who is closest to Obama? for that we use query with model

In [41]:
knn_model.query(obama)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.00169288  | 136.686ms    |
PROGRESS: | 0            | 5602    | 9.4835      | 343.897ms    |
PROGRESS: | 0            | 11164   | 18.8993     | 564.768ms    |
PROGRESS: | 0            | 15380   | 26.0365     | 789.289ms    |
PROGRESS: | 0            | 20757   | 35.1391     | 1.01s        |
PROGRESS: | 0            | 25677   | 43.468      | 1.23s        |
PROGRESS: | 0            | 31202   | 52.8212     | 1.47s        |
PROGRESS: | 0            | 36100   | 61.1129     | 1.68s        |
PROGRESS: | 0            | 40930   | 69.2895     | 1.91s        |
PROGRESS: | 0            | 46050   | 77.957      | 2.13s        |
PROGRESS: | 0            | 50726   | 85.8729     | 2.36s        |
PROGRESS: | 0            | 55514   | 9

query_label,reference_label,distance,rank
0,Barack Obama,0.0,1
0,Joe Biden,0.794117647059,2
0,Joe Lieberman,0.794685990338,3
0,Kelly Ayotte,0.811989100817,4
0,Bill Clinton,0.813852813853,5


As we can see, president Obama's article is closest to the one about his vice-president Biden, and those of other politicians.  

## Other examples of document retrieval

In [42]:
swift = people[people['name'] == 'Taylor Swift']

In [44]:
knn_model.query(swift)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.00169288  | 92.438ms     |
PROGRESS: | 0            | 4012    | 6.79183     | 318.015ms    |
PROGRESS: | 0            | 6950    | 11.7655     | 541.453ms    |
PROGRESS: | 0            | 10953   | 18.5421     | 765.986ms    |
PROGRESS: | 0            | 16173   | 27.3789     | 994.226ms    |
PROGRESS: | 0            | 20450   | 34.6194     | 1.21s        |
PROGRESS: | 0            | 25428   | 43.0465     | 1.45s        |
PROGRESS: | 0            | 30178   | 51.0877     | 1.66s        |
PROGRESS: | 0            | 34714   | 58.7666     | 1.88s        |
PROGRESS: | 0            | 38071   | 64.4496     | 2.11s        |
PROGRESS: | 0            | 42002   | 71.1043     | 2.33s        |
PROGRESS: | 0            | 46054   | 7

query_label,reference_label,distance,rank
0,Taylor Swift,0.0,1
0,Carrie Underwood,0.76231884058,2
0,Alicia Keys,0.764705882353,3
0,Jordin Sparks,0.769633507853,4
0,Leona Lewis,0.776119402985,5


In [30]:
jolie = people[people['name'] == 'Angelina Jolie']

In [31]:
knn_model.query(jolie)

PROGRESS: Starting pairwise querying...
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.00169288  | 24.658ms     |
PROGRESS: | Done         |         | 100         | 149.909ms    |
PROGRESS: +--------------+---------+-------------+--------------+


query_label,reference_label,distance,rank
0,Angelina Jolie,0.0,1
0,Brad Pitt,0.784023668639,2
0,Julianne Moore,0.795857988166,3
0,Billy Bob Thornton,0.803069053708,4
0,George Clooney,0.8046875,5


In [45]:
arnold = people[people['name'] == 'Arnold Schwarzenegger']

In [46]:
knn_model.query(arnold)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.00169288  | 53.147ms     |
PROGRESS: | 0            | 6114    | 10.3503     | 295.705ms    |
PROGRESS: | 0            | 11182   | 18.9298     | 502.157ms    |
PROGRESS: | 0            | 16511   | 27.9511     | 729.308ms    |
PROGRESS: | 0            | 21738   | 36.7998     | 951.159ms    |
PROGRESS: | 0            | 27106   | 45.8872     | 1.19s        |
PROGRESS: | 0            | 32153   | 54.4311     | 1.40s        |
PROGRESS: | 0            | 37346   | 63.2222     | 1.62s        |
PROGRESS: | 0            | 42853   | 72.5449     | 1.84s        |
PROGRESS: | 0            | 47690   | 80.7334     | 2.07s        |
PROGRESS: | 0            | 53085   | 89.8664     | 2.29s        |
PROGRESS: | 0            | 57544   | 9

query_label,reference_label,distance,rank
0,Arnold Schwarzenegger,0.0,1
0,Jesse Ventura,0.818918918919,2
0,John Kitzhaber,0.824615384615,3
0,Lincoln Chafee,0.833876221498,4
0,Anthony Foxx,0.833910034602,5
